#### Trainer provide easily train or fine-tune transformer models
<div>
<img src="image/trainer1.png" width=800>
</div>

#### Preprocessing the GLUE MRPC dataset using dynamic padding

In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
print(raw_datasets.column_names)
print(raw_datasets.num_rows)
print(raw_datasets)
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

{'train': ['sentence1', 'sentence2', 'label', 'idx'], 'validation': ['sentence1', 'sentence2', 'label', 'idx'], 'test': ['sentence1', 'sentence2', 'label', 'idx']}
{'train': 3668, 'validation': 408, 'test': 1725}
DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


/Users/xueqiyao/anaconda3/envs/Pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### Set the model and training arguments before creating the trainer

In [2]:
from transformers import AutoModelForSequenceClassification, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments(
    output_dir="test-trainer",
    per_device_train_batch_size=16,
    per_gpu_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    use_mps_device=True,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/xueqiyao/anaconda3/envs/Pytorch/lib/python3.9/site-packages/transformers/training_args.py:2179: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


#### We can then pass everything to teh ***Trainer*** class and start training

In [3]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer = Trainer()
trainer.train()


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'implements' from 'zope.interface' (/Users/xueqiyao/anaconda3/envs/Pytorch/lib/python3.9/site-packages/zope/interface/__init__.py)